## Semantic clustering of Chinese lexical items

Attempt to determine an ideal way to partition Chinese lexical items by meaning. Then ask: do Chinese semantic components look anything like the resulting system?

In [1]:
import json
import re
import unicodedata

import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats
%matplotlib inline

from sklearn.cluster import KMeans

In [2]:
%load_ext autoreload
%autoreload 2
import util

In [3]:
ids = util.load_ids()

---

Fetch candidate characters for analysis, which have a primary decomposition of semantic + phonetic radical.

In [4]:
two_rads = ids[ids.decomposition.str.len() == 3]
two_rads = pd.concat([two_rads,
                      two_rads.decomposition.apply(lambda x: pd.Series(list(x))).rename(columns={0: "layout", 1: "first", 2: "second"})],
                     axis=1)
two_rads.head()

,character,decomposition,radical,frequency,pinyin,pinyin_toneless,tone,layout,first,second
codepoint,,,,,,,,,,
U+2E80,⺀,⿱丶丶,NaN,None,None,None,None,⿱,丶,丶
U+2EB8,⺸,⿱艹干,NaN,None,None,None,None,⿱,艹,干
U+2EBB,⺻,⿱肀一,NaN,None,None,None,None,⿱,肀,一
U+2EBC,⺼,⿵⺆冫,NaN,None,None,None,None,⿵,⺆,冫
U+3022,〢,⿰丨丨,NaN,None,None,None,None,⿰,丨,丨


In [5]:
analysis_chars = two_rads[two_rads.layout == "⿰"]
analysis_chars.head()

,character,decomposition,radical,frequency,pinyin,pinyin_toneless,tone,layout,first,second
codepoint,,,,,,,,,,
U+3022,〢,⿰丨丨,NaN,None,None,None,None,⿰,丨,丨
U+4E46,乆,⿰𠂈乀,NaN,None,None,None,None,⿰,𠂈,乀
U+4E62,乢,⿰山乚,5.0,None,None,None,None,⿰,山,乚
U+4E63,乣,⿰幺乚,5.0,None,None,None,None,⿰,幺,乚
U+4E68,乨,⿰台乚,5.0,None,None,None,None,⿰,台,乚


-------

## Clustering analysis

In [7]:
import fasttext

In [10]:
ft = fasttext.load_model("cc.zh.300.bin")

MemoryError: std::bad_alloc